In [54]:
import pandas as pd
import seaborn as sns
import sys
sys.path.append('../../src/utils/')
sys.path.append('../../bin/utils')
sns.set_context("talk")
sns.set_theme(style="white")
from Bio import SeqIO
import warnings
import glob
import os
import warnings
warnings.filterwarnings("ignore")

# Structures AF2 
def get_seq(PDBFile):
    with open(PDBFile, 'r') as pdb_file:
        for record in SeqIO.parse(pdb_file, 'pdb-atom'):
            return(str(record.seq))

def get_structure_df(structure_path):
    df = pd.DataFrame(columns=["id", "seq"])
    for structure in structure_path:
        seq = get_seq(structure)
        name = structure.split("/")[-1].split(".")[0]
        # add to dataframe
        df.loc[len(df)] = {"id": name, "seq": seq}
    return(df)

# Real sequence
def get_real_sequence(path_fasta):
    df_references = pd.DataFrame(columns=["id", "seq"])
    for reference in path_fasta:
        # save the sequence and id in dataframe
        for record in SeqIO.parse(reference, "fasta"):
            df_references.loc[len(df_references)] = {"id": record.id, "seq": str(record.seq)}
    return(df_references)

def calculate_percentage_identity(seq1, seq2):
    if len(seq1) != len(seq2):
        percentage_identity = 0

    identical_count = sum(1 for a, b in zip(seq1, seq2) if a == b)

    percentage_identity = (identical_count / len(seq1)) * 100
    return percentage_identity



In [55]:
family = "PF00079"
path_structures = glob.glob("../../../titration_pdb_leila/pred_pdbs/structures/fetched_preprocessed/UniProtKB/id_0.99_cov_1.0/"+family+"*/*.pdb")
path_structures = glob.glob("../../../data/OLD/FETCHED_STRUCTURES/0.9_1/"+family+"*/*.pdb")
path_fasta = glob.glob("../../../titration_fasta/"+family+ "*.fasta")
df_sequences  = get_real_sequence(path_fasta)
df_structures = get_structure_df(path_structures)

In [56]:
df_structures = df_structures.merge(df_sequences, on="id", how="inner", suffixes=("_af2", "_ref"))
df_structures["perc_id"] = df_structures.apply(lambda x: calculate_percentage_identity(x.seq_ref, x.seq_af2), axis=1)

In [52]:
# Check wether there is anything below the said threshold
IDENTITY_THRESHOLD = 0.99
df_structures[df_structures.perc_id < IDENTITY_THRESHOLD].sort_values(by="perc_id", ascending=False)

,id,seq_af2,seq_ref,perc_id
